# 3. Synthetic experiments with Symbolic Pursuit

In this notebook, we shall reproduce one of the experiments from Section 6.1 of the paper.
The idea is to start with a linear pseudo black-box for which the importance vector is known unambiguously and see which interpretability methods identifies this vector the most precisely. Let us start by the useful imports.


In [ ]:
from symbolic_pursuit.models import SymbolicRegressor  # our symbolic model class
from sklearn.metrics import mean_squared_error # we are going to assess the quality of the model based on the generalization MSE
from sympy import init_printing # We use sympy to display mathematical expresssions 
import numpy as np # we use numpy to deal with arrays
import lime 
import lime.lime_tabular
init_printing()

We now define a linear pseudo black-box $f$ defined on a 3 dimensional feature space.

$$ f(x_1,x_2,x_3)= x_1 + 2 \cdot x_2 + 3 \cdot x_3$$ 

The importance vector associated to this model is trivially given by $\beta = (1,2,3)$ In this case, we shall keep it unnormalised, unlike in the main paper as we deal with few examples. Let us translate this in Python. 

In [ ]:
def f(X):
    return X[:, 0]+2*X[:,1]+3*X[:,2]

dim_X = 3

Now draw uniformly 100 test points  that we will feed to a *LIME* explainer <cite data-cite="2480681/WCEBQ7N9"></cite> and to train a Symbolic model.

In [ ]:
n_pts = 100
X = np.random.uniform(0, 1, (n_pts, dim_X))

Now we draw 10 test ponits $x_{test} \equiv U([0,1]^3)$ that we are going to use in order to evaluate the perfomances of both explainers on unseen data.

In [ ]:
n_test = 10
X_test = np.random.uniform(0, 1, (n_test, dim_X))

Since LIME produces importance vectors with entries in the form $(feature \ domain , importance)$ for each feature appearing in decreasing order of importance, we implement a function which identifies the feature from the first entry of the tuple and who sorts the importances in the form $(importance(x_1), importance(x_2), importance(x_3))$.

In [ ]:
def order_weights(exp_list):
    ordered_weights = [0 for _ in range(dim_X)]
    for tup in exp_list:
        feature_id = int(tup[0].split('x_')[1][0])
        ordered_weights[feature_id-1] = tup[1]    
    return ordered_weights    

We are now ready to extract the feature importance for our 10 test points as predicted by the LIME explainer :

In [ ]:
lime_weight_list = []
explainer = lime.lime_tabular.LimeTabularExplainer(X, 
                                                   feature_names=["x_"+str(k) for k in range(1,dim_X+1)], 
                                                   class_names=['f'], 
                                                   verbose=True,
                                                   mode='regression')

for i in range(n_test):
    exp = explainer.explain_instance(X_test[i], f, num_features=dim_X)
    lime_weight_list.append(order_weights(exp.as_list()))  
                            
print(lime_weight_list)    

As we can see from the last output, which is the list of predicted importance vectors, LIME seems to produce a big variety of importance vectors. This is suprising for a global linear model. We also note that the relative importance seem inconsistent with the true importance vector $\beta$ defined above. Let us now train a Symbolic model for $f$ based on our training set.

In [ ]:
symbolic_model = SymbolicRegressor()
symbolic_model.fit(f, X)

We now ask our symbolic model to predict the importance vectors for each test point.

In [ ]:
symbolic_weight_list = [] 
for k in range(n_test):
    symbolic_weight_list.append(symbolic_model.get_feature_importance(X_test[k]))
    

In [ ]:
print(symbolic_weight_list)

As we can see, our results appear to be always consistent and very close to the true importance vector $\beta$.

## References<div class="cite2c-biblio"></div>

<div class="cite2c-biblio"></div>